# EDA espacios Verdes y Comunas

El objetivo principal poder medir y analizar la cantidad y tipo de espacios verdes en la ciudad de buenos aires y a su vez poder compararlos con los que recomiendan organizaciones como la OMS en terminos de M2 (metros cuadrados) de espacios verdes por habitante. Teniendo en cuenta que C.A.B.A. es una de las grandes urbes a nivel mundial.

## Primero se debe instalar geopandas y geoplot

In [1]:
!pip install git+git://github.com/geopandas/geopandas.git
!apt install proj-bin libproj-dev libgeos-dev
!pip install git+git://github.com/ResidentMario/geoplot.git

  Cloning git://github.com/geopandas/geopandas.git to c:\users\tom\appdata\local\temp\pip-req-build-9zc3jltr


  Running command git clone -q git://github.com/geopandas/geopandas.git 'C:\Users\Tom\AppData\Local\Temp\pip-req-build-9zc3jltr'
  ERROR: Error [WinError 2] El sistema no puede encontrar el archivo especificado while executing command git clone -q git://github.com/geopandas/geopandas.git 'C:\Users\Tom\AppData\Local\Temp\pip-req-build-9zc3jltr'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
"apt" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


  Running command git clone -q git://github.com/ResidentMario/geoplot.git 'C:\Users\Tom\AppData\Local\Temp\pip-req-build-qljqaivb'
  ERROR: Error [WinError 2] El sistema no puede encontrar el archivo especificado while executing command git clone -q git://github.com/ResidentMario/geoplot.git 'C:\Users\Tom\AppData\Local\Temp\pip-req-build-qljqaivb'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?



  Cloning git://github.com/ResidentMario/geoplot.git to c:\users\tom\appdata\local\temp\pip-req-build-qljqaivb


In [ ]:
conda install geopandas

In [2]:
import sys
'geopandas' in sys.modules

False

### Se importan bibliotecas

In [3]:
import geopandas as gpd
import geoplot as gplt

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', 1000)  
pd.set_option('display.max_columns', 1000) 

# Carga de datasets

Los datasets a utilizar se pueden encontrar en las siguentes paguinas: 

https://data.buenosaires.gob.ar/dataset/espacios-verdes

https://data.buenosaires.gob.ar/dataset/comunas

El de espacios verdes se descarga tanto con pandas para poder manipularlo como con geopandas para las visualizaciones de los mapas. El de comunas solo con geopandas

In [ ]:
evt = pd.read_csv("datasets/espacios_verdes.csv")

In [ ]:
np.shape(evt)

In [ ]:
evt.head(3)

In [ ]:
evt.info()

In [ ]:
num_rows = evt.shape[0]

In [ ]:
nulls = evt.isnull().sum()

In [ ]:
# Columnas sin nulls
print(nulls[nulls == 0].index.to_list())

In [ ]:
# Columnsas con nulls
print(nulls[nulls != 0].sort_values(ascending=False))

Podemos notar que muchas de las columnas tienen una gran cantidad de nulos y no aportan informacion relevante.

Se procede eliminando las columnas con muchos nulos

### Se depuran las columnas con nulos o irrelevantes

In [ ]:
evt = evt.drop(["nombre_ev", "apadrinada", "observacio", "fuente_geo",
                "fuente_ins", "decreto", "fecha_decr", "ordenanza_", 
                "fecha_orde", "boletin_of", "fecha_bole", "fech_padri", 
                "vig_padri", "alc_conven", "nom_mapa", "DGEV_padri", 
                "mant_2017", "tareas", "estado", "sup_total", "canil", 
                "posta_aero", "etiqueta", "reja", "horario_invierno", 
                "horario_verano", "obs_reja"], axis = 1)

In [ ]:
evt.head(3)

## Uso de geopandas

In [ ]:
ev = gpd.read_file("espacios_verdes.csv")
np.shape(ev)

In [ ]:
ev.head(2)

In [ ]:
com = gpd.read_file("comunasg.csv")
np.shape(com)

In [ ]:
com.head(3)

## Limpieza

Cambiamos los nombres de algunas columnas del dataset de comunas por las siguientes razones:
- La columna "comunas" se cambia por "comuna", para que tenga el mismo nombre que en el dataset de espacios verdes.
- La columna  "area" se cambia por "areac" ya que este es el area de la comuna y no se debe confundir con el dataset de espacios verdes ya que en ese caso "area" indica el area de los espacios verdes.

In [ ]:
com = com.rename(columns = {'comunas': 'comuna', "area":"areac"}, inplace = False)
com.head(3)

In [ ]:
com.info()

En el caso de este datasets son pocas las features y todas utiles

In [ ]:
# Se cambia el formato de la columna comunas a string:
com["comuna"] = com["comuna"].astype("str")
com.head()

# Análisis

Muestro todas las clasificaciones que hay para espacios_verdes

In [ ]:
ev["clasificac"].unique()

In [ ]:
evtxt= evt.groupby(["clasificac"]).area.sum()
evtxt = evtxt.sort_values(ascending=False)

In [ ]:
evtxt

In [ ]:
evtxt.plot(kind = "bar", figsize = (10,5));
plt.title("Suma de area por clasificacion", fontsize=16)
plt.ylabel("Area [millones de M2]", fontsize=14);
plt.xlabel("");

In [ ]:
evtxc= evt.groupby(["comuna"]).area.sum()
evtxc = evtxc.sort_values(ascending=False)

In [ ]:
evtxc

**Se genera un grafico de barras para conocer cuantos M2 de espacios verde tiene cada comuna**

In [ ]:
evtxc.plot(kind="bar", figsize = (10,5))
plt.ylabel("Area [millones de M2]", fontsize=14)
plt.xlabel("Comuna", fontsize=14)
plt.title("Suma de area por comuna", fontsize=16)

que hay en la comuna cero?

In [ ]:
comuna0 = evt.loc[evt["comuna"] == 0]
comuna0

**Se modifican los valores de la comuna por los que realmente son:**

In [ ]:
evt.loc[1497, 'comuna'] = 10.0
evt.loc[1570, 'comuna'] = 8.0
evt.loc[1571, 'comuna'] = 8.0
evt.loc[1573, 'comuna'] = 8.0
evt.loc[1574, 'comuna'] = 8.0
evt.loc[1727, 'comuna'] = 1.0

In [ ]:
# Chequeo de que no quedan comunas 0:
comuna0 = evt["comuna"] == 0
comuna0 = evt[comuna0]
comuna0

In [ ]:
sub_evt = evt.groupby(["comuna", "clasificac"])["area"].sum().unstack()
sub_evt.plot(kind="bar", stacked=True, label=True, figsize = (12,8));
plt.ylabel("Area [millones de M2]", fontsize=14);
plt.xlabel("Comuna", fontsize=14);
plt.title("Suma de area por comuna y clasificacion", fontsize=16);
plt.legend(bbox_to_anchor=(1.3, 1));

Se generan Boxplot por cada clasificacion de Espacios verdes y por comuna

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.boxplot(y="clasificac", x="area", data=evt, fliersize = 6)
plt.title("Distribución de área por clasificación de Espacio Verde", fontsize=16)
plt.xlabel('Area', fontsize=14)
plt.ylabel('')
plt.tight_layout()

Se realiza el mismo grafico exceptuando el parque semipublico ya que tiene parametros muy lejanos a la media (una especie de outlier) y eso dificulta la visualizacion de las demas clasificaciones

In [ ]:
evtsps = evt[evt['clasificac'] != "PARQUE SEMIPÚBLICO"]

plt.figure(figsize=(12,7))
ax = sns.boxplot(y="clasificac", x="area", data=evtsps, fliersize = 6, showfliers=False)
plt.title("Distribución de área por clasificación de Espacio Verde", fontsize=16)
plt.xlabel('Area [m2]', fontsize=14)
plt.ylabel('')
plt.tight_layout()

**Se puede notar facilmente que ademas de "parque semipubluco" las clasificaciones de mayor cantidad de metros cuadrados son los parques, jardines y plazas.**

#### Tambien mostramos un Pairplot clasificado por tipo de Espacio Verde para entender mejor las relaciones de las variables

In [ ]:
sns.pairplot(evt, hue = "clasificac")

plt.show()

Se hacen tablas y gráficos de torta para entender la cantidad de cada espacio y la distribucion de los m2 por clasificacion

In [ ]:
evxcl = evt.clasificac.value_counts()
evxcl

In [ ]:
evxcl.to_list()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw=dict(aspect="equal"))

categorias = evxcl.index.to_list()

data = evxcl.to_list()

wedges, texts = ax.pie(data, wedgeprops=dict(width=0.5), startangle=-40)

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(categorias[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                horizontalalignment=horizontalalignment, **kw)

ax.set_title("Cantidad de Espacios Verdes por Clasificacion", fontsize=16)

percents = [str(round((x/np.array(data).sum())*100, 1))+"%   " + y for x, y in zip(data, categorias)]
plt.legend(wedges, percents, loc='right center', bbox_to_anchor=(1.7, 1.),
           fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.pie(evxcl,labels = evxcl.index,autopct='%1.1f%%')
plt.title('Cantidad de Espacios Verdes por Clasificacion')
plt.axis('equal')
plt.show()

**Tambien se analiza la cantidad distribucion de los metros cuadrados por clasificacion**

In [ ]:
plt.pie(evtxt,labels = evtxt.index,autopct='%1.1f%%',)
plt.title('Metros cuadrados de Espacios Verdes por clasificacion')
plt.axis('equal')
plt.show()

Tanto la tabla como el grafico de tortas se realizan a nivel comuna

In [ ]:
mevxc = evt.groupby(["comuna"]).area.sum()
mevxc

In [ ]:
plt.pie(mevxc,labels = mevxc.index,autopct='%1.1f%%',)
plt.title('Cantidad de Espacios Verdes por Clasificacion')
plt.axis('equal')
plt.show()

In [ ]:
ordenevt = evt[evt.clasificac == "PARQUE" ].sort_values('area',ascending=False) 
ordenevt

## Geopandas

In [ ]:
ev = ev.drop(["nombre_ev", "apadrinada", "observacio", "fuente_geo", "fuente_ins", "decreto", "fecha_decr", "ordenanza_", "fecha_orde", "boletin_of", "fecha_bole", "fech_padri", "vig_padri", "alc_conven", "nom_mapa", "DGEV_padri", "mant_2017", "tareas", "estado", "sup_total", "canil", "posta_aero", "etiqueta", "reja", "horario_invierno", "horario_verano", "obs_reja"], axis = 1)

In [ ]:
evt.loc[1497, 'comuna'] = 10.0
evt.loc[1570, 'comuna'] = 8.0
evt.loc[1571, 'comuna'] = 8.0
evt.loc[1573, 'comuna'] = 8.0
evt.loc[1574, 'comuna'] = 8.0
evt.loc[1727, 'comuna'] = 1.0

### Se continuo realizando gráficos de mapas para entender la distribucion y clasificacion de los distintos espacios verdes en la ciudad de buenos aires

Grafico de espacios verdes en la ciudad segun su clasificacion

In [ ]:
gplt.choropleth(ev, hue = ev['clasificac'], figsize = (20,16), legend = True)

Grafico de espacios verdes, con escala de colores segun su area

In [ ]:
naranja = 'OrRd'
gplt.choropleth(ev, hue = ev['area'], cmap = naranja, figsize = (20,16))

Se probaron distintas formas para graficar lo mismo

In [ ]:
ev.plot(figsize = (20,16))

In [ ]:
ev.plot(column='clasificac', legend=True, figsize= (16,12))

Se puede notar que los parques, plazoletas y plazas son los predominantes en la ciudad

tambien se separo por espacios con mayor area total para poder visualizarlo mejor, la leyenda indica area total

In [ ]:
clasif = ev.dissolve(by='clasificac', aggfunc='sum')
clasif.plot(column = 'area', scheme='quantiles', legend=True, figsize = (15,11));

Luego se grafico solo un tipo de espacios verdes para entender la distribucion de cada uno

PLAZAS

In [ ]:
ax = ev[ev.clasificac == 'PLAZA'].plot(color='white', edgecolor='black')
ax.plot(ax=ax, color='red', figsize = (18,12))
plt.show()

PLAZOLETAS

In [ ]:
ax = ev[ev.clasificac == 'PLAZOLETA'].plot(color='white', edgecolor='black')
ax.plot(ax=ax, color='red', figsize = (18,12))
plt.show()

PARQUES

In [ ]:
ax = ev[ev.clasificac == 'PARQUE'].plot(color='white', edgecolor='black')
ax.plot(ax=ax, color='red', figsize = (18,12))
plt.show()

Se puede notar una gran cantidad de plazas y plazoletas bastante distribuidas a lo largo de la ciudad, sin embargo parece los parques parecen no estar del todo distribuidos, dejando zonas lejos de alguno

### Se procedio a analizar las comunas

In [ ]:
gplt.choropleth(com, hue = com['comuna'], figsize = (20,16))

## Concatenar datasets

Se unio la coluna de area de espacios verdes al dataset de comuna, recordar:
- "area" equivale a area de espacios verdes
- "areac" equivale al area total de la comuna

Se convierte el formato de la columna en string

In [ ]:
mevxc = evt.groupby(["comuna"]).area.sum()
mevxc = mevxc.reset_index()
mevxc["comuna"] = mevxc["comuna"].astype("int")
mevxc["comuna"] = mevxc["comuna"].astype("str")
mevxc.head()


Se concatenan los datos

In [ ]:
evxcom = com.merge(mevxc, left_on="comuna", right_on="comuna", how='inner')
evxcom.head()

Se genera el gráfico degradado segun cantidad de M2 de espacios verdes por comuna

In [ ]:
gplt.choropleth(evxcom, hue = evxcom['area'], cmap = 'OrRd', figsize = (20,16), legend = True)

## Conclusion parcial

Creeamos que si bien los datasets no contiene muchas features cuantificables, con los datos de areas de espacios verdes y formas geometricas tanto de espacios como comunas se pueden analisis utiles y especificos. Ademas se tiene pensado unirlo con mas datasets de densidad poblacional en cada comuna de la ciudad y con el financiamiento que cada una recibe, para hacer un analisis aun mas global.

# Bibliografía

Se utilizaron distintas paguinas para contender el manejo de datasets de caracter geografico, sumado a las librerias basicas



https://towardsdatascience.com/plotting-geographical-data-with-geopandas-338cc7e17e4e

https://geopandas.org/aggregation_with_dissolve.html

https://geopandas.org/aggregation_with_dissolve.html
https://www.datacamp.com/community/tutorials/geospatial-data-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034364&utm_targetid=dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1000073&gclid=Cj0KCQjwt4X8BRCPARIsABmcnOrWMSIaHzSoPH6EUAqAK2TPpsrLjKSR6wy41-uzoi5kBgnQXT49jBgaAuOgEALw_wcB



# EDA Población

Para un complemento al análisis se obtuvieron datos acerca de la demografía de la Ciudad con datos desde el 2015.

### Data extraida de la página del Gobierno de la Ciudad

Links: https://data.buenosaires.gob.ar/dataset/estructura-poblacion

In [ ]:
poblacion = pd.read_excel(r'datasets\Estimacion_Poblacion_nuevo.xlsx', delimiter=',', parse_dates = True)

In [ ]:
poblacion.AÑO.unique()

In [ ]:
np.shape(poblacion)

In [ ]:
poblacion.head()

In [ ]:
poblacion.columns

In [ ]:
col_nulls = poblacion.isnull().any()
col_nulls

In [ ]:
comunas_renames = dict([(x, 'Comuna ' + str(x)) for x in range(1, 16)])

poblacion.rename(columns={'Grupo de edad (años)':'Rango Etario'},inplace=True) 
poblacion.rename(columns=comunas_renames,inplace=True)

In [ ]:
poblacion.columns

In [ ]:
poblacion.head()

In [ ]:
index_pobl=poblacion[poblacion ['Rango Etario'] == 'Total'].index

In [ ]:
poblacion.drop(index_pobl , inplace=True)

In [ ]:
poblacion.head()

In [ ]:
# Mayor registro de población por comuna
np.max(poblacion.loc[:, 'Comuna 1':'Comuna 15'])

### Nos quedamos con el estimado de población del año 2020

In [ ]:
pobl_2020 = poblacion.loc[poblacion.AÑO == 2020]

In [ ]:
pobl_2020.head(5)

In [ ]:
# total de la población para 2020
total_poblacion = pobl_2020['Total'].sum()
f_string = f'Total estimado de la población en CABA para el 2020: {total_poblacion:,} personas'.replace(',','.')
print(f_string)

## Total por género

In [ ]:
pivot_2020 = pd.pivot_table(data = pobl_2020, values = ['Total'], index = ['SEXO'], aggfunc= np.sum) 

In [ ]:
pivot_2020

In [ ]:
pivot_2020.plot(kind='pie', subplots=True, figsize=(12, 6), autopct='%1.1f%%')

## Total por comuna y género

In [ ]:
pivot_comuna_2020 = pd.pivot_table(data = pobl_2020, index = ['SEXO'], values= ['Comuna 1', 'Comuna 2', 'Comuna 3', 'Comuna 4',
       'Comuna 5', 'Comuna 6', 'Comuna 7', 'Comuna 8', 'Comuna 9', 'Comuna 10',
       'Comuna 11', 'Comuna 12', 'Comuna 13', 'Comuna 14', 'Comuna 15'] ,aggfunc= np.sum) 

In [ ]:
pivot_comuna_2020

In [ ]:
pivot_comuna_2020.columns

In [ ]:
sns.barplot(x='SEXO',y='Total',data=pobl_2020,palette="rocket")
plt.title("Distribución población por género", fontsize=14)

# Conclusión parcial:

# EDA presupuestos

Al igual que el análisis de Población este de Presupuestos se utiliza como un complemento del análisis de Espacios Verdes. 
Se busca obtener insights que relacionen la cantidad presupuestada a los proyectos de espacios verdes con la calidad de los mismos por comuna

Links: https://data.buenosaires.gob.ar/dataset/presupuesto-sancionado

In [ ]:
%run Funciones_Auxiliares.ipynb

# Lectura de la base unificada y selección de los campos a analizar

La base contiene todos los gastos que demande el desenvolvimiento de los órganos del gobierno central, de los entes descentralizados y comunas, el servicio de la deuda pública, las inversiones patrimoniales y los recursos para cubrir tales erogaciones. Los recursos publicados detallan la distribución de los créditos aprobados al máximo nivel de desagregación.

De todas las áreas querremos analizar la destinada a espacios verdes.

In [ ]:
presupuesto_sancionado = pd.read_csv("datasets/presupuesto_sancionado_clean.csv")

In [ ]:
presupuesto_sancionado['SANCION'] = presupuesto_sancionado['SANCION'].astype(float)

In [ ]:
presupuesto_sancionado = presupuesto_sancionado.loc[presupuesto_sancionado['SANCION'] != 0]
presupuesto_sancionado = presupuesto_sancionado.loc[presupuesto_sancionado['AÑO'] > 2009]

In [ ]:
presupuesto_sancionado.AÑO.unique()

In [ ]:
presupuesto_sancionado.shape

In [ ]:
cols = presupuesto_sancionado.dtypes[(presupuesto_sancionado.dtypes == 'object')].index
for col in cols:
    presupuesto_sancionado[col] = presupuesto_sancionado[col].astype(str)

**El Ministerio de Ambiente pasa a ser una secretaria en 2020 bajo la dependencia de la Jefatura de Gobierno**

In [ ]:
jef_gob = presupuesto_sancionado.groupby('JURISDICCION').get_group("JEFATURA DE GOBIERNO")
sec_ambiente = jef_gob.groupby('SUBJURISDICCION').get_group("SECRETARIA DE AMBIENTE")
min_ambiente = presupuesto_sancionado.groupby('JURISDICCION').get_group("MINISTERIO DE AMBIENTE Y ESPACIO PUBLICO")

ambiente = pd.concat([min_ambiente, sec_ambiente])

In [ ]:
ambiente['JURISDICCION'].value_counts()

In [ ]:
ambiente['FUNCION'].value_counts()

In [ ]:
ecologia = ambiente[ambiente['FUNCION'] == 'ECOLOGIA']
ecologia = ecologia.T.drop_duplicates().T  # Elimino columnas que contienen la misma información.

In [ ]:
ecologia.shape

In [ ]:
def repr_high_values(series):

        series = series.apply(lambda x: f"${x:,.0f}" if not pd.isnull(x) else x)
        series = series.str.replace(",", ".")

        return series

In [ ]:
def repr_sancion_by_subset(df, subset):

    sancion_by_subset = df.groupby(subset).agg({'SANCION': 'sum'})\
                                .sort_values(by='SANCION', ascending=False)
    
    sancion_by_subset['SANCION_repr'] = repr_high_values(sancion_by_subset['SANCION'])
    
    return sancion_by_subset

In [ ]:
ecologia['SANCION'] = ecologia['SANCION'].astype(float)

In [ ]:
sancion_por_unidad_ejecutora_año = ecologia.pivot_table(index='AÑO', columns='UNIDAD_EJECUTORA', values='SANCION', aggfunc='sum')
sancion_por_unidad_ejecutora_año.loc['Total'] = sancion_por_unidad_ejecutora_año.apply('sum', axis=0) 
sancion_por_unidad_ejecutora_año = sancion_por_unidad_ejecutora_año.T.sort_values(by='Total', ascending=False).T

sancion_por_unidad_ejecutora_año = sancion_por_unidad_ejecutora_año.apply(repr_high_values, axis=1).fillna('-')

sancion_por_unidad_ejecutora_año

DIRECCION GENERAL DE ESPACIOS VERDES: https://www.buenosaires.gob.ar/gestioncomunal/institucional-subsecretaria-de-gestion-comunal/direccion-general-de

In [ ]:
sancion_por_unidad_ejecutora_año[["DIRECCION GENERAL DE ESPACIOS VERDES", 'DIRECCION GENERAL DE ARBOLADO']]

**Los datos sobre la dirección general de arbolado están desactualizados.**

**Y Falta información del 2020 sobre espacios verdes**

### Acá filtramos la base para espacios verdes

Los datos de arbolados en principio no se usarán por estar desactualizados

In [ ]:
espacios_verdes = ecologia[ecologia['UNIDAD_EJECUTORA'] == 'DIRECCION GENERAL DE ESPACIOS VERDES']

In [ ]:
espacios_verdes.shape

### Sanción de presupuesto por año para Espacios Verdes

In [ ]:
sancion_por_unidad_ejecutora_año[["DIRECCION GENERAL DE ESPACIOS VERDES"]].T

In [ ]:
sancion_espacios_verdes_año = espacios_verdes.groupby('AÑO').agg({'SANCION': 'sum'})['SANCION']
g = sancion_espacios_verdes_año.plot(kind='bar', figsize=(12, 6));

ylabels = ['$ {:,.0f}'.format(x) + ' M' for x in g.get_yticks()/1000000];
g.set_yticklabels(ylabels);
plt.title("Sanción de presupuesto por año para Espacios Verdes", fontsize=18);

### Pasaremos los datos de presupuestos a valores del 2010 para ver si realmente aumentó la inversión en el área

In [ ]:
# http://estudiodelamo.com/inflacion-argentina-anual-mensual-2020/

header = ["AÑO","PORC_ANUAL", "ENE","FEB","MAR","ABR","MAY","JUN","JUL","AGO","SEP","OCT","NOV","DIC"]
inflacion = pd.DataFrame(([ 2011,  23.97, 2.10, 1.50, 2.10, 1.80, 1.33, 1.56, 1.66, 1.85, 1.82, 1.52, 1.46, 1.86],
                          [ 2012,  25.98, 1.84, 1.79, 2.27, 2.19, 1.81, 1.63, 1.80, 1.93, 1.92, 1.72, 1.81, 2.04],
                          [ 2013,  23.28, 2.61, 1.14, 1.50, 1.43, 1.63, 2.19, 2.63, 2.14, 2.12, 2.11, 2.38, 3.30],
                          [ 2014,  38.53, 4.62, 4.31, 3.42, 2.59, 2.28, 2.20, 2.47, 2.65, 2.48, 2.25, 1.86, 1.87],
                          [ 2015,  27.50, 2.08, 1.48, 2.12, 2.01, 2.00, 1.53, 1.92, 2.17, 1.92, 1.52, 2.20, 3.80],
                          [ 2016,  40.30, 3.60, 4.80, 3.20, 6.70, 3.50, 2.90, 2.40, 0.50, 0.80, 2.90, 1.90, 1.60],
                          [ 2017,  24.80, 1.60, 2.10, 2.20, 2.10, 1.80, 1.30, 2.10, 1.50, 1.60, 1.50, 1.40, 3.10],
                          [ 2018,  47.65, 1.76, 2.42, 2.34, 2.74, 2.08, 3.74, 3.10, 3.89, 6.53, 5.39, 3.15, 2.57],
                          [ 2019,  53.83, 2.91, 3.77, 4.68, 3.44, 3.06, 2.72, 2.20, 3.95, 5.89, 3.29, 4.25, 3.74],
                          [ 2020,  36.62, 2.25, 2.01, 3.34, 1.50, 1.54, 2.24, 1.93, 2.70, 2.84,]), columns=header)

inflacion

In [ ]:
inflacion_anual = inflacion.set_index('AÑO')['PORC_ANUAL'] / 100
inflacion_anual

In [ ]:
def presupuestos_a_valores_2010(sancion_espacios_verdes_año):
    presupuestos_llevados_2010 = []
    for start_year, presupuesto in sancion_espacios_verdes_año.items():
#         print('Pesos presupuestados finales del año {}:  $ {:,.0f}'.format(start_year, presupuesto))

        for tasa_inflacion in inflacion_anual.loc[start_year::-1]:

            presupuesto = presupuesto / (1+tasa_inflacion)

        presupuestos_llevados_2010.append(presupuesto)    
#         print("Pesos finales llevados a principios 2010:   $ {:,.0f}\n".format(presupuesto))

    presupuestos_llevados_2010 = pd.Series(presupuestos_llevados_2010)
    presupuestos_llevados_2010.index = sancion_espacios_verdes_año.index
    
    return presupuestos_llevados_2010

In [ ]:
presupuestos_llevados_2010 = presupuestos_a_valores_2010(sancion_espacios_verdes_año)

aux = pd.concat([presupuestos_llevados_2010, sancion_espacios_verdes_año], axis=1)
aux.columns = ['presupuestos llevados a pesos del 2010', 'presupuestos sancionados para espacios verdes']
g = aux.plot(figsize=(12,6))

ylabels = ['$ {:,.0f}'.format(x) + ' M' for x in g.get_yticks()/1000000];
g.set_yticklabels(ylabels);
plt.title("Sanción de presupuesto por año para Espacios Verdes", fontsize=18);

In [ ]:
presupuestos_llevados_2010 = presupuestos_llevados_2010.reset_index()
presupuestos_llevados_2010.columns = ['AÑO', 'SANCION']
presupuestos_llevados_2010['SANCION_repr'] = repr_high_values(presupuestos_llevados_2010['SANCION'])

In [ ]:
presupuestos_llevados_2010 = presupuestos_llevados_2010.set_index('AÑO')
presupuestos_llevados_2010.T

In [ ]:
g = presupuestos_llevados_2010.plot(figsize=(8, 6))

ylabels = ['$ {:,.0f}'.format(x) + ' M' for x in g.get_yticks()/1000000];
g.set_yticklabels(ylabels);
plt.title("Sanción de presupuesto llevado a valores $ del 2010", fontsize=14);
plt.grid(True);

**Se puede observar que la inversión real en el área viene bajando año a año con picos en los años impares**

**Esta subida inferimos que viene asociada a años electorales**

### Sanciones por Proyecto, Actividad y Obra.

In [ ]:
repr_sancion_by_subset(espacios_verdes, subset='PROYECTO')

In [ ]:
repr_sancion_by_subset(espacios_verdes, subset='ACTIVIDAD')

In [ ]:
repr_sancion_by_subset(espacios_verdes, subset='OBRA')

In [ ]:
# Sunburst que etiqueta los proyectos
sunburst(espacios_verdes)

### Sanción por Comuna

In [ ]:
sancion_espacios_verdes = repr_sancion_by_subset(espacios_verdes, subset=['AÑO', 'UBICACION_GEOGRAFICA']).reset_index()

In [ ]:
sancion_espacios_verdes

In [ ]:
fig = px.bar(sancion_espacios_verdes, x="AÑO", y="SANCION", color="UBICACION_GEOGRAFICA",
             title="<b>Sanción por Comuna<b>")
fig = fig.update_layout(
    title={
        'y':0.9,
        'x':0.135,
        'xanchor': 'left',
        'yanchor': 'top'})

fig = fig.update_layout(yaxis_tickformat = '$,.')

fig.show()

**Análisis del gráfico de barras:**

1. **Comuna 1:**  Sigue tendencia creciente
9. **Comuna 9:**  Sigue tendencia creciente
6. **Comuna 6:**  Sigue tendencia creciente
12. **Comuna 12:** Sigue tendencia creciente
13. **Comuna 13:** Sigue tendencia creciente
15. **Comuna 15:** Sigue tendencia creciente


2. **Comuna 2:**  Revisar datos. En 2016 hay una caida muy fuerte.
7. **Comuna 7:**  Caida en 2016
8. **Comuna 8:**  Brusca caida 2016 y 2017. Pico fuerte en 2019
10. **Comuna 10:** Brusca caida 2016 y 2017. Pico fuerte en 2019



3. **Comuna 3:**  Picos de inversión muy fuertes en 2014 y 2019
4. **Comuna 4:**  Pico de inversión muy fuerte en 2019

5. **Comuna 5:**  Mayor inversión en 2014, cae mucho en 2016.
11. **Comuna 11:** Inversión hasta 2015 y luego cae bruscamente y queda en niveles bajos.
14. **Comuna 14:** Inversión hasta 2017, cae en los últimos años



### Búsqueda de match por categoría de espacios verdes

In [ ]:
clasificaciones_espacios_verdes = ['PLAZOLETA', 'PARQUE', 'PLAZA', 'CANTERO', 'JARDIN',
                                   'PASEO', 'PATIO',
                                   'BARRIO', 'COMPLEJO', 'JARDIN BOTANICO',]

In [ ]:
a = espacios_verdes['ACTIVIDAD'].unique()
a[['PARQUE' in x for x in a]]

**plazoleta, barrio, complejo, cantero central . No se encuentran en espacios verdes pero hay referencias en otras categorías.**

In [ ]:
for clas in clasificaciones_espacios_verdes:
 
    for col in espacios_verdes.loc[:, 'PROGRAMA':'OBRA'].columns:
        
        print(col, clas, espacios_verdes[col].str.contains(clas).sum())
        print(espacios_verdes.loc[espacios_verdes[col].str.contains(clas), col].unique())

        print()

**Queda como posibilidad mergear por categoría de espacio verde. Pero en proncipio se unirá por comuna**

# Conclusión Parcial

### MERGE